# Notes on Machine Learning Models - Part 1 - Feature Engineering

အပိုင်း ၃ ပိုင်းရှိတဲ့ ဒီ notes တွေမှာ အဓိကအားဖြင့် Machine Learning Model တွေကို အသုံးပြုရာမှာ သတိပြုရမဲ့ အောက်ပါ အကြောင်းအရာများကို ပြောပြပေးသွားမှာ ဖြစ်ပါတယ်။ 

* Feature Engineering 
* Model Validation and
* Coding Best Practices

ဒီအပိုင်းက ပထမဆုံးအပိုင်း Feature Engineering ဖြစ်ပါတယ်။ 

$X$ ရော $y$ ပါရှိတဲ့ **supervised** Machine Learning ပဲ ဖြစ်ဖြစ်၊ $X$ တမျိုးပဲ ရှိတဲ့ **unsupervised** Machine Learning ပဲ ဖြစ်ဖြစ် Feature တွေ ရွေးချယ်မှုနဲ့ Engineering လုပ်တာတွေဟာ အရေးပါပါတယ်။ 

ဒါကြောင့် ဒီအပိုင်း မှာ 

* Feature Encoding/Scaling/Normalization
* Text/Image/Time-series Manipulation
* Feature Selection Techniques တွေနဲ့ 
* Dimensionality Reduction/Increment Techniques တွေ အကြောင်း ပြောမယ်။

## Feature Encoding/Scaling/Normalization

In [ ]:
from sklearn import preprocessing as sk_pp
from sklearn import datasets as sk_ds

import numpy as np
import pandas as pd

In [ ]:
df_X, ds_y = sk_ds.fetch_openml(name="credit-g", as_frame=True, return_X_y=True)
df_X.head()

### Missing Values


ဒီ dataset မှာ null value `NaN` ပါ/မပါကို အရင် ကြည့်ရမယ်။ ကြည့်ပုံကြည့်နည်းကတော့ `pandas.DataFrame` ရဲ့ `isna` method နဲ့ပါပဲ။

In [ ]:
df_X.isna().any()

In [ ]:
ds_y.isna().any()

ဒီတော့ ဒီ data မှာ null value `NaN` မပါတာကို တွေ့ရမယ်။ ပါခဲ့ရင် `pandas.DataFrame.fillna` method နဲ့ ဖြည့်တာ (သို့မဟုတ်) `pandas.DataFrame.dropna` နဲ့ ဖြုတ်ချတာ တခုခု လုပ်ရမယ်။

### Feature Encoding


`sklearn` ပဲ ဖြစ်ဖြစ်၊ `XGBoost` ပဲ ဖြစ်ဖြစ်၊ `lightgbm` ပဲ ဖြစ်ဖြစ် ... အခု ခေတ်စားနေတဲ့ Deep Learning `pytorch`/`tensorflow` ပဲ ဖြစ်ဖြစ် numeric value မဟုတ်ရင် ဘာမှ လုပ်လို့ မရဘူး။ ဒါကြောင့် Numeric Value မဟုတ်တဲ့ Categorical Feature တွေကို Encode လုပ်ပေးရတယ်။

ဒီလို encode လုပ်ရာမှာ 

* One-Hot Encoding နဲ့ 
* Ordinal Encoding (Label Encoding) ဆိုပြီး ၂ မျိုး ရှိတယ်။

#### Knowing `dtypes`
ဘာပဲဖြစ်ဖြစ် encode မလုပ်ခင်မှာ ဘယ် column တွေက ဘာ data type လဲဆိုတာ သိဖို့လိုတယ်။ 

In [ ]:
df_X.dtypes

data type ပေါ်မူတည်ပြီး column တွေကို ရွေးနိုင်ဖို့ `pandas.DataFrame.select_dtypes` method ကို သုံးနိုင်တယ်။

In [ ]:
df_X_category = df_X.select_dtypes(include=["category"])
df_X_category.head()

In [ ]:
df_X_number = df_X.select_dtypes(include=["number"])
df_X_number.head()

#### Ensuring data nature

Data nature ဆိုတာ categorical လား၊ numeric လားကို ဆိုလိုတယ်။ 

လက်တွေ့ဘဝမှာ computer ထဲ သိမ်းထားတဲ့ `dtypes` နဲ့ data nature က အမြဲတူမနေနိုင်ဘူး။

တချို့ dataset တွေမှာ numeric value ဖြစ်နေပေမဲ့ categorical data ဖြစ်နေတတ်တယ်။ 

> ဥပမာ McDonald's က အရောင်း transaction ဆိုရင် Combo 1, Combo 2, Combo 3 ကို 1, 2, 3 လို့ encode လုပ်ထားတဲ့ column ပါလာတာမျိုး။ ဒါကြောင့် some numbers may be categories လို့ အမြဲသတိထားရမယ်

ပြီးတော့ ... 

* Categorical မှာမှ order ရှိတဲ့ ordinal (XS/S/M/L/XL/XXL) လား၊ order မရှိတဲ့ norminal (blue/green/red) လား ထပ်ခွဲနိုင်ပြီး

* Numeric မှာမှ integer နဲ့ float ဆိုပြီး ထပ်ခွဲနိုင်တယ်။

ဒါတွေကို သိနိုင်ဖို့ `pandas.Series.unique` method နဲ့ `pandas.Series.nunique` method တွေကို သုံးပြီး ကြည့်နိုင်တယ်။

In [ ]:
col_to_unique_values = {c: [df_X[c].unique()] for c in df_X.columns}
df_col_to_unique_values = pd.DataFrame(data=col_to_unique_values, index=["unique_values"]).T
df_col_to_unique_values.loc[:, "unique_count"] = df_col_to_unique_values.apply(lambda x : df_X[x.name].nunique(), axis=1)
df_col_to_unique_values

အထူးသဖြင့် `unique_count` နည်းတဲ့ numeric column တွေကို အဓိကထားပြီး ကြည့်ရမယ်။ 

> ဒီ dataset မှာတော့ categorical nature နဲ့ numeric column မရှိဘူး။ ဥပမာ ... `residence_since` ဆိုရင် 4.0 က 2.0 ထက် ကြီးတယ်။ categorical မဟုတ်ဘူး။ 

တခါတလေမှာ `pandas.Series.unique` အစား `pandas.Series.value_counts` function ကို သုံးနိုင်တယ်။

In [ ]:
ds_y.value_counts()

#### Encoding


အလွယ်အားဖြင့် ordinal data တွေကို ordinal encode လုပ်ပြီး norminal data တွေကို one-hot encode လုပ်ရမယ်။

အခု ဒါတွေကို သိအောင် data ကို explore လုပ်ရမယ်။

In [ ]:
for c in df_X_category.columns:
    print ("Column : {}".format(c))
    print (df_X_category[c].dtype.categories)

ဒီနေရာမှာ မြင်သာတဲ့ ordinal data တွေက အောက်ပါတို့ ဖြစ်တယ်။ သူတို့နဲ့အတူ အစီအစဉ်ကိုပါ ပြထားတယ်။

* `credit_history` : `['no credits/all paid', 'all paid', 'existing paid', 'delayed previously', 'critical/other existing credit']`
* `savings_status` : `['no known savings', '<100', '100<=X<500', '500<=X<1000', '>=1000']` -- ဒီနေရာမှာ ပြန်စီလိုက်တာကို သတိပြုပါ

တခြား ordinal data တွေ ရှိနေနိုင်သေးပေမဲ့ ကျန်တာကို ဒီနေ့အဖို့ one-hot encoding ပဲ လုပ်လိုက်မယ်။

> `personal_status` ကနေပြီး `gender` ထုတ်လို့ ရတာကို သတိထားပါ။ အိမ်စာအနေနဲ့ ထုတ်ကြည့်ပါ။

In [ ]:
df_features = pd.DataFrame(index=df_X.index, data=None)

ordinal_columns = ["credit_history", "savings_status"]
oe = sk_pp.OrdinalEncoder(
    # အောက်က categories parameter မှာ array-like of array-like (list of list) ထည့်ပေးရတာ သတိပြုပါ။
    categories=[
        ['no credits/all paid', 'all paid', 'existing paid', 'delayed previously', 'critical/other existing credit'],
        ['no known savings', '<100', '100<=X<500', '500<=X<1000', '>=1000']
    ], 
    # handle_unknown က သိပ်အရေးကြီးတယ်။ ဒါမပါသွားရင် production ကျမှ ပြဿနာ တက်တတ်တယ်။ default is "error"
    handle_unknown="use_encoded_value", 
    unknown_value=np.nan
)

df_features.loc[:, ["oe_{}".format(c) for c in ordinal_columns]] = oe.fit_transform(df_X[ordinal_columns])
df_features.head()

**Err** အပေါ်က cell က code မှာ သီအိုရီ/သဘောတရား အမှားတခု ပါနေတယ်။ 

> စာသင်ချိန်အတွင်းမှာပဲ ရအောင် ရှာပါ။

In [ ]:
from sklearn import model_selection as sk_ms

df_X_tr, df_X_ts, ds_y_tr, ds_y_ts = sk_ms.train_test_split(df_X, ds_y, test_size=0.2, shuffle=True, random_state=42)

df_feat_tr = pd.DataFrame(data=None, index=df_X_tr.index)
df_feat_ts = pd.DataFrame(data=None, index=df_X_ts.index)

In [ ]:
ordinal_columns = ["credit_history", "savings_status"]
oe = sk_pp.OrdinalEncoder(
    # အောက်က categories parameter မှာ array-like of array-like (list of list) ထည့်ပေးရတာ သတိပြုပါ။
    categories=[
        ['no credits/all paid', 'all paid', 'existing paid', 'delayed previously', 'critical/other existing credit'],
        ['no known savings', '<100', '100<=X<500', '500<=X<1000', '>=1000']
    ], 
    # handle_unknown က သိပ်အရေးကြီးတယ်။ ဒါမပါသွားရင် production ကျမှ ပြဿနာ တက်တတ်တယ်။ default is "error"
    handle_unknown="use_encoded_value", 
    unknown_value=np.nan
)

df_feat_tr.loc[:, ["oe_{}".format(c) for c in ordinal_columns]] = oe.fit_transform(df_X_tr[ordinal_columns])
# see ? you can never ever fit with whole dataset; 
df_feat_ts.loc[:, ["oe_{}".format(c) for c in ordinal_columns]] = oe.transform(df_X_ts[ordinal_columns])

df_feat_tr.head()

In [ ]:
norminal_columns = [c for c in df_X_category.columns if c not in ordinal_columns]

ohe = sk_pp.OneHotEncoder(sparse=False, handle_unknown="ignore")
ohe.fit(df_X_tr[norminal_columns])

norminal_features = ohe.get_feature_names_out()
df_feat_tr.loc[:, norminal_features] = ohe.transform(df_X_tr[norminal_columns])
df_feat_ts.loc[:, norminal_features] = ohe.transform(df_X_ts[norminal_columns])
df_feat_tr.head()

In [1]:
df_feat_ts.head()

NameError: name 'df_feat_ts' is not defined